Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [8]:
print(gender_series:= df['gender'].value_counts(), 
      '\n', '*'*50, 
      f"\nMale\t{round(gender_series['Male']/sum(gender_series)*100, 2)} %",
      f"\nFemale\t{round(gender_series['Female']/sum(gender_series)*100, 2)} %",
      '\n', '*'*50,)

Male      3555
Female    3488
Name: gender, dtype: int64 
 ************************************************** 
Male	50.48 % 
Female	49.52 % 
 **************************************************


##### 2. Какое количество уникальных значений у поля InternetService?

In [9]:
print(len(df['InternetService'].value_counts())) 

3


##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [10]:
TotalCharges_series = df.loc[df['TotalCharges'] != ' ', 'TotalCharges'].astype(float).copy()
print(f"Медиана\t\t\t{TotalCharges_series.median():.2f}", 
      f"\nСреднее\t\t\t{TotalCharges_series.mean():.2f}",
      f"\nСтандартное отклонение\t{TotalCharges_series.std():.2f}")

Медиана			1397.47 
Среднее			2283.30 
Стандартное отклонение	2266.77


В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [14]:
import time
check = df['PhoneService'].value_counts().values
result_dict = {}
df_copy = df.copy()
start = time.time()
df_copy.loc[df_copy['PhoneService'] == 'Yes', 'PhoneService'] = 1
df_copy.loc[df_copy['PhoneService'] == 'No', 'PhoneService'] = 0
result_dict['v2'] = time.time() - start
print(df_copy['PhoneService'].value_counts().values == check)

[ True  True]


##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [15]:
df_copy = df.copy()
df_copy.loc[df_copy['TotalCharges']==' ', 'TotalCharges'] = np.nan
df_copy['TotalCharges'] = df_copy['TotalCharges'].astype(np.float32)
df_copy = df_copy.fillna(0)
df_copy.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692,2279.732178
std,0.368612,24.559481,30.090047,2266.794434
min,0.000000,0.000000,18.250000,0.000000
25%,0.000000,9.000000,35.500000,398.549988
50%,0.000000,29.000000,70.350000,1394.550049
75%,0.000000,55.000000,89.850000,3786.599976
max,1.000000,72.000000,118.750000,8684.799805


##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [16]:
df_copy = df.copy()
df_copy.loc[df_copy['Churn'] == 'Yes', 'Churn'] = 1
df_copy.loc[df_copy['Churn'] == 'No', 'Churn'] = 0
df_copy['Churn'].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [17]:
def convert(d, c, v): 
    for key, new_value in v.items():
      d.loc[d[c] == key, c] = new_value
    return d

df_copy = df.copy()
columns_list = ['StreamingMovies', 'StreamingTV', 'TechSupport']
for c in columns_list:
  print(df_copy[c].value_counts(), '\n', '*'*100)

for c in columns_list:
  df_copy = convert(df_copy, c, {'Yes': 1, 'No': 0, 'No internet service': 0})

for c in columns_list:
  print(df_copy[c].value_counts(), '\n', '*'*100)

No                     2785
Yes                    2732
No internet service    1526
Name: StreamingMovies, dtype: int64 
 ****************************************************************************************************
No                     2810
Yes                    2707
No internet service    1526
Name: StreamingTV, dtype: int64 
 ****************************************************************************************************
No                     3473
Yes                    2044
No internet service    1526
Name: TechSupport, dtype: int64 
 ****************************************************************************************************
0    4311
1    2732
Name: StreamingMovies, dtype: int64 
 ****************************************************************************************************
0    4336
1    2707
Name: StreamingTV, dtype: int64 
 ****************************************************************************************************
0    4999
1    2044
Name:

##### 8. Заполните пропуски в поле PhoneService значением 0

In [ ]:
#Ваш код здесь

#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [18]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']
df_copy = df.copy()
df_copy[columns].head(3)

,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,Female,1,No,29.85,No,No,No,No
1,Male,34,Yes,1889.5,No,No,No,No
2,Male,2,Yes,108.15,No,No,No,Yes


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [19]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size = 0.3)


##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [20]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [21]:
gender.fit_transform(X_train)


,gender_Female,gender_Male
3419,0,1
5844,1,0
915,0,1
1622,1,0
5540,1,0
...,...,...
2487,0,1
2009,0,1
6916,0,1
2062,1,0


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [24]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [25]:
from sklearn.preprocessing import StandardScaler

TotalCharges =  Pipeline([
                ('selector', NumberSelector(key='TotalCharges')),
                ('standard', StandardScaler())
            ])

X_train = X_train.loc[X_train['TotalCharges']!=' ']
TotalCharges.fit_transform(X_train)


array([[ 1.82683554],
       [-0.94977518],
       [-0.9854369 ],
       ...,
       [-0.65783186],
       [-0.27876421],
       [-0.30922481]])

Объединение всех "кубиков" очень легко сделать таким образом

In [26]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [27]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [29]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

ValueError: could not convert string to float: 'Yes'

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [23]:
print('Колличество классов', pipeline.predict_proba(X_test).shape[1])


NameError: name 'pipeline' is not defined

##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [30]:
from sklearn.metrics import roc_auc_score, log_loss

print(f"AUC-ROC:\t{roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1]):.5f}",
      f"\nlog loss:\t{log_loss(y_test, pipeline.predict_proba(X_test)[:, 1]):.5f}")

ValueError: could not convert string to float: ''

### Сохраним наш пайплайн

In [31]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)

ModuleNotFoundError: No module named 'dill'